In [ ]:
import warnings

warnings.filterwarnings("ignore")

## IRIS Dataset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# Load IRIS from SKLearn
from sklearn.datasets import load_iris

In [ ]:
iris = load_iris()
iris.keys()

In [ ]:
iris.feature_names

In [ ]:
iris.target

In [ ]:
iris_df = pd.DataFrame(iris.data)
iris_df.columns = ["sepal_length", "sepal_width", "petal_length", "petal_width"]
iris_df["target"] = iris.target

In [ ]:
def decode_labels(y_idx):
    return iris.target_names[y_idx]

In [ ]:
# Plant Species
iris_df.target.value_counts()

In [ ]:
iris_df.sample(10)

## Creating Torch Dataset

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

In [ ]:
class IRISDataset(Dataset):
    
    def __init__(self, data, y=None):
        self.x = data  # Add .astype(np.float32) to convert to Flat Tensor
        self.y = y
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        return self.x[idx], self.y[idx]

In [ ]:
feature_cols = ["sepal_length", "sepal_width", "petal_length", "petal_width"]

train_ds = IRISDataset(iris_df[feature_cols].values, iris_df["target"].values)

In [ ]:
# Take a look at a sample
next(iter(train_ds))

In [ ]:
# Now that Dataset is Defined we can create a DataLoader to feed batches
train_dl = DataLoader(train_ds, batch_size=16, shuffle=True)

## Build the Model

In [ ]:
# We have Four Inputs (Sepal Length, width and Petal Length and width) and three possible outputs (0, 1, 2)
net = nn.Sequential(nn.Linear(4, 3))

In [ ]:
net

In [ ]:
for inputs, labels in train_dl:
    net(inputs)

In [ ]:
## You will find these errors to be frunstrating sometimes, but at least they are Python Errors.
## X Inputs need to be Float and not Double Tensors, but np.float64, gets converted to double tensor

## Run the Model

* Select Optimizer and Loss Function
* Run forward and backward pass for a number of epochs

In [ ]:
optim = torch.optim.SGD(net.parameters(), lr=0.001)

loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
def train(net, train_dl, nb_epochs=50, verbose=1):
    epoch_losses = []

    # Training Loop
    for epoch in range(nb_epochs):
        net.train()
        losses = []
        for x, y in train_dl:
            # Run the forward pass
            outputs = net(x)

            # Get Loss Value
            loss = loss_fn(outputs, y)
            losses.append(loss.item())

            # Zero Out the Gradient
            optim.zero_grad()

            # Run a Backward Pass
            loss.backward()

            # Update the Gradients
            optim.step()

        epoch_losses.append(np.mean(losses))
        if verbose:
            print("Epoch {} Loss: {:.4f}".format(epoch, np.mean(losses)))
        
    return epoch_losses

In [ ]:
losses = train(net, train_dl, nb_epochs=50)

In [ ]:
plt.plot(losses)

### Add more Layers

1. Define the Net
2. Create Optimizer, Loss
3. Run the Training Loop

In [ ]:
## Your turn
net = nn.Sequential(nn.Linear(4, 8),
                   nn.Linear(8, 3))

In [ ]:
optim = torch.optim.SGD(net.parameters(), lr=0.01)

loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
losses = train(net, train_dl, nb_epochs=500, verbose=0)

In [ ]:
plt.plot(losses)

In [ ]:
## That's a lot of boilerplate code.  Lots of people have created their own functions

In [ ]:
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import CategoricalAccuracy, Loss

from utils import log_training_loss, log_training_results

In [ ]:
lr = 1e-2
momentum = 0.9

In [ ]:
model = nn.Sequential(nn.Linear(4, 8),
                   nn.Linear(8, 3))

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
loss_fn = torch.nn.CrossEntropyLoss()

metrics = {
    'accuracy': CategoricalAccuracy(),
    'loss': Loss(loss_fn)
}
trainer = create_supervised_trainer(model, optimizer, loss_fn, device=device)

evaluator = create_supervised_evaluator(model,
                                        metrics=metrics,
                                        device=device)

In [ ]:
trainer.run(train_dl, max_epochs=5)

In [ ]:
trainer.add_event_handler(Events.ITERATION_COMPLETED, log_training_loss, 
                           data_loader=train_dl, 
                           log_interval=10)

trainer.add_event_handler(Events.EPOCH_COMPLETED, log_training_results, 
                           data_loader=train_dl, 
                           metrics=metrics, 
                          is_train=True)

In [ ]:
trainer.run(train_dl, max_epochs=50)

In [ ]:
# Predict
model(torch.tensor([0.4, 0.3, 0.2, 0.1]))